# Course 3: Localization
## Part 2: Markov Localization in C++
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Derive and implement the [Bayes filter](https://en.wikipedia.org/wiki/Recursive_Bayesian_estimation) for localisation;
* Approximate vehicle location as a probability distribution, then perform probabilistic reasoning over time (filtering);
* Use the [Markov Assumption](https://en.wikipedia.org/wiki/Markov_chain) and [law of total probability](https://en.wikipedia.org/wiki/Law_of_total_probability) to initialise the filter with meaningful estimates.

## 1. Introduction

In this part of the Markov Localization lesson we will be programming in C++. In order to follow along with the code in this Jupyter notebook, a C++ kernel will need to be installed. We use the [`xeus-cling`](https://github.com/jupyter-xeus/xeus) kernel, which you can access via [this MyBinder-hosted](https://mybinder.org/v2/gh/QuantStack/xeus-cling/stable?filepath=notebooks/xcpp.ipynb) notebook instance.

## 2. Programming Task

### 2.1. Initialize Priors Function

In this task we will create a function that initialises priors (initial belief state for each position on the map) given landmark positions, a position standard deviation ($\pm 1.0$), and the assumption that our car is parked next to a landmark.

Note that the control standard deviation represents the spread from movement (movement is the result of our control input in this case). We input a control of moving $1.0$ step but our actual movement could be in the range of $\pm 1.0$ control standard deviation. The position standard deviation is the spread in our actual position. For example, we may believe start at a particular location, but we could be anywhere in that location $\pm$ our position standard deviation.

In [1]:
// From J. Moran's `1_initialize_priors.cc`

In [2]:
#include <iostream>
#include <vector>


/* Calculates the 1-D location posterior, i.e., the vector of prior probabilty
 * values for each position in the discretised 1-D pose range space (map) with
 * step resolution of 1.0m. The range space is defined relative to the heading
 * of the ego-vehicle. Here we assume the vehicle starts at one of the
 * `n_landmarks` (static objects) with a position std. deviation of +/- 1.0m.
 */
std::vector<float> initialize_priors(
    int map_size,
    std::vector<float> landmark_positions,
    float position_stdev
) {
  // Initialise the prior probabilities
  std::vector<float> priors(map_size, 0.0);
  // Number of total landmarks in map view
  int n_landmarks = landmark_positions.size();
  // Number of neighbours for each landmark
  int n_neighbours =  2 * int(position_stdev);
  // Number of non-zero priors (landmarks plus neighbours)
  int n_pos = n_landmarks + n_neighbours * n_landmarks;
  // The prior probability of vehicle being at a position of interest
  float p_prior = 1.0 / n_pos;
  // Set the non-zero prior probability values
  for (auto i : landmark_positions) {
      // The landmark prior probability
      priors[int(i)] = p_prior;
      // The neighbouring positions' prior probabilities
      priors[int(i - position_stdev)] = p_prior;  // Left of landmark
      priors[int(i + position_stdev)] = p_prior;  // Right of landmark
  }
  return priors;
}

#### Testing the initialize priors function

For simplicity we assumed a position standard deviation
```cpp
position_stdev = 1.0f
```
and coded a solution for initialising priors accordingly.

Expected result:

```pseudo
0
0
0
0
0.111111
0.111111
0.111111
0
0
0.111111
0.111111
0.111111
0
0
0
0
0
0
0
0.111111
0.111111
0.111111
0
0
0
```

In [3]:
// From J. Moran's `1_tests.cc`

In [4]:
/* Tests the `initialize_priors` function inside `1_initialize_priors`.
 * Returns the initialised 1-D prior probability vector.
 */
void test_initialize_priors() {
  // Set standard deviation of position
  float position_stdev = 1.0f;
  // Set map horizon distance in meters
  int map_size = 25;
  // Initialise landmarks
  std::vector<float> landmark_positions{5, 10, 20};
  // Testing initialise priors function
  std::vector<float> priors = initialize_priors(map_size,
                                                landmark_positions,
                                                position_stdev
  );
  // Print the probability values to `stdout`
  for (int p = 0; p < priors.size(); ++p) {
    std::cout << priors[p] << "\n";
  }
}

In [5]:
test_initialize_priors()

0
0
0
0
0.111111
0.111111
0.111111
0
0
0.111111
0.111111
0.111111
0
0
0
0
0
0
0
0.111111
0.111111
0.111111
0
0
0


Great — our results seem to match the expected! ✅

Note that a more robust approach would be to create a function that accepts a position standard deviation as an argument and adjusts initialisation and normalisation based on its value.

### 2.2. Determine Probabilities

Here we practise using `normpdf` in this workspace, setting a `value` to maximise the probability when given a parameter of `1.0`.

In [6]:
// From J. Moran's `2_determine_probabilities.cc`

In [7]:
#include <cassert>
#include <math.h>

// Define the normalisation term
const static float kOneOverSqrt2Pi = 1.0 / sqrt(2 * M_PI);


class Helpers {
public:
  /* Returns the probability function for value `x` assuming a
   * normal distribution parameterised by (`mu`, `sigma`).
   */
  static float normpdf(float x, float mu, float std) {
    // Distribution parameters must be scalars
    // and std must be positive
    assert(std > 0.0);
    // Compute the probability function
    return (kOneOverSqrt2Pi / std) * exp(-0.5 * pow((x - mu) / std, 2));
  }
};

#### Testing determine probabilities function

Here we select a position
```cpp
value = 1.0
```
which maximises the control parameter when `parameter = 1.0`.

In [8]:
// From J. Moran's `1_tests.cc`

In [9]:
/* Tests the `normpdf` function inside `2_determine_probabilities`.
 * Returns the probability vector from the normal distribution
 * parameterised with the given scalar values of (`mu`, `sigma`).
 */
void test_determine_probabilities() {
  // The position `x`
  float value = 1.0;
  // The position / observation control parameter
  float parameter = 1.0;    // Number of units moved each time-step
  // The position / observation standard deviation
  float stdev = 1.0;
  // Compute the probability distribution for the given values
  static float prob = Helpers::normpdf(value, parameter, stdev);
  // Print the returned probability value
  std::cout << prob << std::endl;
}

In [10]:
test_determine_probabilities()

0.398942


The resulting probability value will always be maximised when the control `parameter` and position `value` are equal. Generally speaking, we should observe a maximum probability value at:

```cpp
x = value - control
```

Note that we assume here a control parameter of `1.0` to represent the number of moves per unit of time. Therefore, the vehicle is assumed to have moved `1.0` units per time-step.

### 2.3. Motion Model Probability II

In the previous exercises in [`2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb`]() we manually executed the steps for determining pseudo-range estimates and the overall observation model probability. Now, let's implement a function that accepts a vector of landmark positions — the pseudo positions `x`, that returns a vector of sorted (ascending) pseudo-range estimates. The output vector of this function will be later used as input to our observation model function.

To implement the `pseudo_range_estimator` function, we must do the following for each pseudo-position $x_{t}^{(i)}$:

1. Loop over all landmark positions:
    * (a) Calculate the distance between each pseudo-position $x_{t}^{(i)}$ and the given landmark;
    * (b) Append the distance to the pseudo-range vector if its value is positive (i.e., the landmark is in front of the pseudo-position);
2. Return the resulting pseudo-range vector, sorted in ascending order.

Note that the resulting pseudo-range vector may not contain all values from the input `x`. That is because we assume not all values of `x` have landmark positions in front of the vehicle's current position. 

In [1]:
// From J. Moran's `3_get_pseudo_ranges.cc`

In [10]:
#include <algorithm>
#include <vector>

/* Returns the pseudo-range estimates for a given set
* of landmark positions. The positions are defined
* in 1-D global map coordinates with respect to the
* heading of the ego-vehicle (i.e., the forward motion).
*/
std::vector<float> pseudo_range_estimator(
  std::vector<float> landmark_positions,
  float pseudo_position
) {
  // Initialise the pseudo-observation vector and distance
  std::vector<float> pseudo_ranges(0.0, landmark_positions.size());
  float dist = 0.0;
  // 1. Loop over landmarks
  for (auto l_pos : landmark_positions) {
    // 1a. Compute the distance between position and landmark
    dist = l_pos - pseudo_position;
    if (dist > 0.0) {
      // Landmark is in front of pseudo-position, append to list
      pseudo_ranges.push_back(dist);
    }
    else {
      // Skip the landmark, could be behind vehicle
      continue;
    }
  }
  // Sort the resulting vector in ascending order
  std::sort(pseudo_ranges.begin(), pseudo_ranges.end());
  return pseudo_ranges;
}

#### Testing the pseudo-ranges function

In [11]:
// From J. Moran's `1_tests.cc`

In [12]:
#include <iostream>
#include <vector>


/* Tests the `pseudo_range_estimator` function inside `3_get_pseudo_ranges.cc`.
 * Returns the pseudo-range vector containing the landmark distances relative
 * to the given `pseudo_position`.
 */
void test_pseudo_range_estimator() {
  // The landmark positions in 1-D map space
  std::vector<float> landmark_positions{5, 10, 12, 20};
  // The number of discrete positions `x` on the map
  int map_size = 25;
  // Number of metres moved by the vehicle per time-step
  float movement_per_timestep = 1.0f;
  // The standard deviation of the control
  float control_stdev = 1.0f;
  // Compute the pseudo-ranges for each position on the map
  for (int i = 0; i < map_size; i++) {
    float pseudo_position = float(i);
    std::vector<float> pseudo_ranges = pseudo_range_estimator(landmark_positions,
                                                              pseudo_position
    );
    // Print the resulting pseudo-range vector values
    if (pseudo_ranges.size()) {
      for (auto val : pseudo_ranges) {
        std::cout << "x: " << i << "\t" << val << "\n";
      }
      std::cout << "-----------------------" << "\n";
    }
    else {
      // No landmarks in front of vehicle
      continue;
    }
  }
}

In [13]:
test_pseudo_range_estimator()

x: 0	5
x: 0	10
x: 0	12
x: 0	20
-----------------------
x: 1	4
x: 1	9
x: 1	11
x: 1	19
-----------------------
x: 2	3
x: 2	8
x: 2	10
x: 2	18
-----------------------
x: 3	2
x: 3	7
x: 3	9
x: 3	17
-----------------------
x: 4	1
x: 4	6
x: 4	8
x: 4	16
-----------------------
x: 5	5
x: 5	7
x: 5	15
-----------------------
x: 6	4
x: 6	6
x: 6	14
-----------------------
x: 7	3
x: 7	5
x: 7	13
-----------------------
x: 8	2
x: 8	4
x: 8	12
-----------------------
x: 9	1
x: 9	3
x: 9	11
-----------------------
x: 10	2
x: 10	10
-----------------------
x: 11	1
x: 11	9
-----------------------
x: 12	8
-----------------------
x: 13	7
-----------------------
x: 14	6
-----------------------
x: 15	5
-----------------------
x: 16	4
-----------------------
x: 17	3
-----------------------
x: 18	2
-----------------------
x: 19	1
-----------------------


Fantastic — Our results match the expected output! 🎉

## Credits

This assignment was prepared by Aaron Brown, Tiffany Huang and Maximilian Muffert of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).